In [1]:
import csv
import re
import multiprocessing as mp
import pandas as pd
from datasets import load_dataset, load_from_disk
import numpy as np

## 提取question_info

In [2]:
question_dic = {}
now_index = 0

question_infos = []

# 先找到所有python问题，他们的问题数量以及他们是否有最佳答案
with open("./generated_file/q_temp_output.csv", "r") as q_f:
    reader = csv.DictReader(q_f)
    for row in reader:
        
        target_tag = "python"
        # 判断target_tag在不在里面
        tags = row['Tags']
        tags = re.sub(r"[<>]", " ", tags).split()
        if target_tag not in tags:
            continue
        
        # 判断有没有最佳答案，只要有的
        AcceptedAnswerId = row["AcceptedAnswerId"]

        if AcceptedAnswerId == "":
            continue
        else:
            f_best_answer = AcceptedAnswerId
        
        # 只有一个答案？都可以
        answer_count = eval(row["AnswerCount"])
        if answer_count < 2:
            pass

        q_id = row["Id"]
        temp = [q_id]

        question_dic[q_id] = now_index
        now_index += 1

        temp.append(f_best_answer)
        temp.append(row["AnswerCount"])
        question_infos.append(temp)
        
        if now_index % 10000 == 0:
            print(f"\rget {now_index} questions!", end="")

print()
question_number = now_index
print("finally get: ", question_number, " questions!")


get 150000 questions!

KeyboardInterrupt: 

In [19]:
answers = []
for i in range(question_number):
    answers.append([])

with open("./generated_file/a_temp_output.csv", "r") as a_f:
    reader = csv.DictReader(a_f)
    for row in reader:
        q_id = row["ParentId"]
        # a_id, score
        temp = [row['Id'], row["Score"]]

        if question_dic.get(q_id) is None:
            continue
        answers[question_dic[q_id]].append(temp)


# 把信息整合存起来
f_out = open("generated_file/python_question_info", "w")

for i in range(question_number):

    # q_id + 1 + answer_count + best_answer_id + answers ...
    write_info = question_infos[i][0] + " 1 " + question_infos[i][2] + " " + question_infos[i][1]
    for item in answers[i]:
        if item[0] == question_infos[i][1]:
            continue
        write_info += " " + item[0]
    write_info += "\n"

    f_out.write(write_info)

f_out.close()

In [4]:
with open("generated_file/python_question_info", "r") as f:
    count = 0
    for line in f:
        count += 1
    print(count)

840133


# 获得qa的内容

In [3]:
q_ids = []
a_ids = []

with open("generated_file/python_question_info", "r") as f:
    for index, line in enumerate(f):
        
        # q_id + 1 + answer_count + best_answer_id + answers ...
        infos = line.split()
        answer_num = len(infos) - 3

        # 没有最佳答案
        if eval(infos[1]) != 1:
            continue
            
        temp_q_id = infos[0]
        temp_a_ids = infos[3:]
        
        q_ids.append(eval(temp_q_id))
        
        for a in temp_a_ids:
            a_ids.append(eval(a))

q_ids.sort()
a_ids.sort()


In [ ]:
content_out = open("generated_file/python_q_id_content" + ".csv", "w", newline='')

csv_head = ["id", "title", "body"]

out_writer = csv.writer(content_out)
out_writer.writerow(csv_head)

q_csv_reader = pd.read_csv('generated_file/q_temp_output.csv', index_col=0, chunksize=1000)
q_df = next(q_csv_reader)


# 开始查表，为了找到body, title, tags
for target_id in q_ids:

        if q_df.index[0] > target_id:
            continue
            
        while q_df.index[-1] < target_id:
            q_df = next(q_csv_reader)
        
        try:
            temp_z = q_df.loc[target_id]
            body, title, tags = temp_z["Body"], temp_z["Title"], temp_z["Tags"]
        except KeyError:
            continue

        if title == None or body == None:
            continue

        out_writer.writerow([target_id, title, body])

content_out.close()

In [ ]:
content_out = open("generated_file/python_a_id_content" + ".csv", "w", newline='')

csv_head = ["id", "body"]

out_writer = csv.writer(content_out)
out_writer.writerow(csv_head)

a_csv_reader = pd.read_csv('generated_file/a_temp_output.csv', index_col=0, chunksize=1000)
a_df = next(a_csv_reader)


# 开始查表，为了找到body, title, tags
for target_id in a_ids:

        if a_df.index[0] > target_id:
            continue
            
        while a_df.index[-1] < target_id:
            a_df = next(a_csv_reader)
        
        try:
            temp_z = a_df.loc[target_id]
            body = temp_z["Body"]
        except KeyError:
            continue

        if body is None:
            continue

        out_writer.writerow([target_id, body])

print("finished!")

content_out.close()

# 把qa内容处理一下,同时生成corpus

In [23]:
out_file = open("./glove/qa_corpus", "w")

In [24]:
# 先处理问题
dataset = load_dataset('csv', data_files='generated_file/python_q_id_content.csv')['train']

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def map_process_data(rows):
    global out_file
    global REPLACE_BY_SPACE_RE
    global GOOD_SYMBOLS_RE

    old_body = rows['body']
    old_title = rows['title']
    
    new_body = []
    new_title = []
    temp_string = ""

    for index, body in enumerate(old_body):
        
        # 处理body
        text = body
        text = re.sub(r"<pre><code>.*?</code></pre>", " HONOCODE ", text, flags=re.DOTALL)
        # 然后把尖括号里的东西都删掉
        text = re.sub(r"<[^>]*>", " ", text, flags=re.DOTALL)
        text = text.lower()
        text = REPLACE_BY_SPACE_RE.sub(' ', text)
        text = GOOD_SYMBOLS_RE.sub('', text)
        
        new_body.append(text)
        temp_string += text + " "

        # 处理title
        text = old_title[index]
        text = re.sub(r"<pre><code>.*?</code></pre>", " HONOCODE ", text, flags=re.DOTALL)
        # 然后把尖括号里的东西都删掉
        text = re.sub(r"<[^>]*>", " ", text, flags=re.DOTALL)
        text = text.lower()
        text = REPLACE_BY_SPACE_RE.sub(' ', text)
        text = GOOD_SYMBOLS_RE.sub('', text)
        
        new_title.append(text)
        temp_string += text + "\n"

    out_file.write(temp_string)
    return {'title':new_title, 'body':new_body}

dataset = dataset.map(function=map_process_data, batched=True, with_indices=False)

Using custom data configuration default-59bdbd4d2207c2c3


Dataset csv downloaded and prepared to /home/yuanhang/.cache/huggingface/datasets/csv/default-59bdbd4d2207c2c3/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [25]:
# 存成csv格式
content_out = open('generated_file/processed_python_q_id_content.csv', "w", newline='')

out_writer = csv.writer(content_out)

csv_head = ["id", "title", "body"]
out_writer.writerow(csv_head)

written_id = dataset['id']
written_body = dataset['body']
written_title = dataset['title']

for index, temp_id in enumerate(written_id):
    out_writer.writerow([temp_id, written_title[index], written_body[index]])

content_out.close()

In [26]:
out_file.close()
out_file = open("./glove/qa_corpus", "a")

In [27]:
# 处理答案
dataset = load_dataset('csv', data_files='generated_file/python_a_id_content.csv')['train']

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def map_process_data(rows):
    global out_file
    global REPLACE_BY_SPACE_RE
    global GOOD_SYMBOLS_RE

    old_body = rows['body']
    
    new_body = []
    temp_string = ""

    for index, body in enumerate(old_body):
        # 处理body
        text = body
        text = re.sub(r"<pre><code>.*?</code></pre>", " HONOCODE ", text, flags=re.DOTALL)
        # 然后把尖括号里的东西都删掉
        text = re.sub(r"<[^>]*>", " ", text, flags=re.DOTALL)
        text = text.lower()
        text = REPLACE_BY_SPACE_RE.sub(' ', text)
        text = GOOD_SYMBOLS_RE.sub('', text)
        
        new_body.append(text)
        temp_string += text + "\n"

    out_file.write(temp_string)
    return {'body':new_body}

dataset = dataset.map(function=map_process_data, batched=True, with_indices=False)

Using custom data configuration default-96040d8102891915


Dataset csv downloaded and prepared to /home/yuanhang/.cache/huggingface/datasets/csv/default-96040d8102891915/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [28]:
# 存成csv格式
content_out = open('generated_file/processed_python_a_id_content.csv', "w", newline='')

out_writer = csv.writer(content_out)

csv_head = ["id", "body"]
out_writer.writerow(csv_head)

written_id = dataset['id']
written_body = dataset['body']

for index, temp_id in enumerate(written_id):
    out_writer.writerow([temp_id, written_body[index]])

content_out.close()

In [29]:
out_file.close()

# 把qa内容轻微处理一下，用来做问卷

In [4]:
# 先处理问题
dataset = load_dataset('csv', data_files='generated_file/python_q_id_content.csv')['train']

Using custom data configuration default-59bdbd4d2207c2c3
Reusing dataset csv (/home/yuanhang/.cache/huggingface/datasets/csv/default-59bdbd4d2207c2c3/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [9]:
print(dataset)
print(dataset['body'][100])

Dataset({
    features: ['id', 'title', 'body'],
    num_rows: 840133
})
<p>Does Python have a unit testing framework compatible with the standard xUnit style of test framework? If so, what is it, where is it, and is it any good?</p>



In [10]:
def map_process_data(rows):
    old_body = rows['body']
    old_title = rows['title']
    
    new_body = []
    new_title = []
    
    for index, body in enumerate(old_body):
        
        # 处理body
        text = body
        text = re.sub(r"<[^>]*>", " ", text, flags=re.DOTALL)
        new_body.append(text)


        # 处理title
        text = old_title[index]
        text = re.sub(r"<[^>]*>", " ", text, flags=re.DOTALL)
        new_title.append(text)
        
    return {'title':new_title, 'body':new_body}

In [11]:
dataset = dataset.map(function=map_process_data, batched=True, with_indices=False)

In [12]:
print(dataset)
print(dataset['body'][100])

Dataset({
    features: ['body', 'id', 'title'],
    num_rows: 840133
})
 Does Python have a unit testing framework compatible with the standard xUnit style of test framework? If so, what is it, where is it, and is it any good? 



In [14]:
# 存成csv格式
content_out = open('generated_file/natural_python_q_id_content.csv', "w", newline='')

out_writer = csv.writer(content_out)

csv_head = ["id", "title", "body"]
out_writer.writerow(csv_head)

written_id = dataset['id']
written_body = dataset['body']
written_title = dataset['title']

for index, temp_id in enumerate(written_id):
    out_writer.writerow([temp_id, written_title[index], written_body[index]])

content_out.close()

In [15]:
# 处理答案
dataset = load_dataset('csv', data_files='generated_file/python_a_id_content.csv')['train']

def map_process_data(rows):
    
    old_body = rows['body']
    new_body = []

    for index, body in enumerate(old_body):
        # 处理body
        text = body
        # 然后把尖括号里的东西都删掉
        text = re.sub(r"<[^>]*>", " ", text, flags=re.DOTALL)
        new_body.append(text)

    return {'body':new_body}



Using custom data configuration default-96040d8102891915
Reusing dataset csv (/home/yuanhang/.cache/huggingface/datasets/csv/default-96040d8102891915/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [16]:
dataset = dataset.map(function=map_process_data, batched=True, with_indices=False)

In [17]:
# 存成csv格式
content_out = open('generated_file/natural_python_a_id_content.csv', "w", newline='')

out_writer = csv.writer(content_out)

csv_head = ["id", "body"]
out_writer.writerow(csv_head)

written_id = dataset['id']
written_body = dataset['body']

for index, temp_id in enumerate(written_id):
    out_writer.writerow([temp_id, written_body[index]])

content_out.close()

# 计算idf

In [2]:
# 首先要统计每个词的idf, 不包括<unk>
# 建立字典, 包括一些特殊字符
my_word2index = {'<pad>': 0, '<unk>': 1, '<sos>': 2, '<eos>': 3, '<cls>': 4}
now_index = 5

with open("/data/yuanhang/important_code/so_python/glove/vocab.txt", "r") as f:
    for line in f:
        temp = line.split()
        word = temp[0]
        if my_word2index.get(word) is None:
            my_word2index[word] = now_index
            now_index += 1

my_index2word = {v:k for k, v in my_word2index.items()}

# voc size
print(now_index)

129519


In [3]:
# 统计每个词在文档中的出现次数
word_appear_count = []
document_count = 0

for i in range(0, now_index):
    word_appear_count.append(0)

with open("/data/yuanhang/important_code/so_python/glove/qa_corpus", "r") as f:
    for line in f:
        # 去重复
        words = set(line.split())
        
        if len(words) > 0:
            document_count += 1
            
        for word in words:
            index = my_word2index.get(word)
            if index is not None:
                word_appear_count[index] += 1

In [13]:
print(len(word_appear_count))
print(word_appear_count[100])

129519
187397


In [14]:
# 计算idf
word_idf = []
for i in range(0, now_index):
    word_idf.append(0)

max_idf = -1
min_idf = document_count
zero_count = 0
zero_indices = []

for index, value in enumerate(word_appear_count):
    if value == 0:
        zero_count += 1
        zero_indices.append(index)
    else:
        idf = np.log(document_count/value)
        word_idf[index] = idf

        if idf < min_idf:
            min_idf = idf
        if idf > max_idf:
            max_idf = idf

In [15]:
print(min_idf, max_idf)

print(zero_count)
for index in zero_indices:
    print(my_index2word[index])

0.22294190666590855 14.674845678961981
5
<pad>
<unk>
<sos>
<eos>
<cls>


In [17]:
# 把 my_word2index 和 word_idf 存起来
np.save('./generated_file/my_word2index.npy', my_word2index)

with open("./generated_file/word_idf", 'w') as f:
    for idf in word_idf:
        f.write(str(idf)+'\n')

# 整理下question_info,过滤掉有问题的

In [18]:
dataset = load_from_disk("./generated_file/python_qa_info.dataset")

In [19]:
dataset

Dataset({
    features: ['a_ids', 'q_id'],
    num_rows: 840128
})

In [20]:
dataset[0]

{'q_id': 337,
 'a_ids': [342,
  471,
  525,
  635,
  69410,
  69772,
  123307,
  199213,
  202259,
  7954780,
  13832269,
  23143835]}

In [21]:
q_id_content = pd.read_csv("generated_file/processed_python_q_id_content.csv", index_col=['id'])
a_id_content = pd.read_csv("generated_file/processed_python_a_id_content.csv", index_col=['id'])

/home/yuanhang/anaconda3/envs/fd/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [22]:
q_id_content.iloc[0]
a_id_content.iloc[0]

body     personally  ive played with several of the bu...
Name: 342, dtype: object

In [23]:
# 用来存有价值的id
my_q_ids = []
my_answer_ids = []

with open("generated_file/python_question_info") as f:
    # q_id + 1 + answer_count + best_answer_id + answers ...
    for line in f:
        result = line.split()
        
        temp_q_id = eval(result[0])
        temp_a_ids = []
        
        for temp_a_id in result[3:]:
            temp_a_ids.append(eval(temp_a_id))
        
        # 判断这个问题是不是空的
        q_title = q_id_content.loc[temp_q_id]['title']
        
        try:
            best_answer_body = a_id_content.loc[temp_a_ids[0]]['body']
        except KeyError:
            continue
        
        if q_title is not None and best_answer_body is not None:
            my_q_ids.append(temp_q_id)  
            my_answer_ids.append(temp_a_ids)

In [24]:
print(len(my_q_ids))

840128


In [ ]:
my_qa_dict = {'q_id':my_q_ids, 'a_ids':my_answer_ids}
my_qa_dict = Dataset.from_dict(my_qa_dict)
my_qa_dict.save_to_disk("./generated_file/python_qa_info.dataset")

## 找相关问题，见temp_code